In [12]:
from sklearn.ensemble import *
import sklearn.metrics
import pandas as pd
import numpy as np

In [2]:
def load_file_data(path):
    file = open(path, 'r')
    data = []
    for line in file:
        data.append(line.strip("\n").split())
    return np.array(data)

def load_embedding(path, skipFirst = True):
    file = open(path, 'r')
    ids = []
    data = []
    for line in file:
        if skipFirst:
            skipFirst = False
            continue
        embedding = line.strip("\n").split()
        data.append(embedding)
    return np.array(data).astype('float64')

In [3]:
dataset = 'fb15k'
train = load_file_data('data/'+dataset+'/train.txt')
test = load_file_data('data/'+dataset+'/test.txt')
valid = load_file_data('data/'+dataset+'/valid.txt')

embeddings = load_embedding('data/'+dataset+'/embedding.txt')

In [4]:
def x_y_split(data, embeddings):
    embedding_frame = pd.DataFrame(embeddings, dtype='float64')
    x_cols = embedding_frame.columns.values[1:]
    embedding_frame[0] = embedding_frame[0].astype('int64')
    
    data_frame = pd.DataFrame(data=data, columns=['subject', 'object', 'relation'], dtype=np.int32)
    counts = data_frame.groupby('subject').count()['relation']

    y = counts.sort_index()
    x = embedding_frame[embedding_frame[0].isin(data_frame['subject'].unique())].sort_values(by=0).reset_index(drop=True)[x_cols]
    
    return x, y

In [15]:
# TODO: This doesn't work... we need to count the number of edges in the whole dataset and then split it...
x_train, y_train = x_y_split(train, embeddings)
x_test, y_test = x_y_split(test, embeddings)
x_valid, y_valid = x_y_split(valid, embeddings)

x_train = pd.concat([x_train, x_valid])
y_train = pd.concat([y_train, y_valid])

In [17]:
regressors = [
    ('RandomForest', RandomForestRegressor(20, max_depth=2, random_state=0)),
    ('AdaBoost', AdaBoostRegressor(random_state=0)),
    ('GradientBoost', GradientBoostingRegressor(random_state=0))
]

for name, regr in regressors:
    regr.fit(x_train, y_train)
    print(name, sklearn.metrics.mean_squared_error(y_test, regr.predict(x_test)))

RandomForest 325.66836068773836
AdaBoost 1173.4906017139929
GradientBoost 379.880298928149
